In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
# Read the TSV file into a DataFrame
df = pd.read_csv('correct_twitter_201904.tsv', delimiter='\t', low_memory=False)

# Clean the data if needed (e.g., handling NaN values)
df.fillna('', inplace=True)

# Convert to lowercase for uniformity
df['text'] = df['text'].str.lower()

# Create a SQLite database and store the DataFrame
conn = sqlite3.connect('twitter_data.db')
df.to_sql('tweets', conn, if_exists='replace', index=False)

88037

In [3]:
df.head()

,id,event,ts1,ts2,from_stream,directly_from_stream,from_search,directly_from_search,from_quote_search,directly_from_quote_search,...,retweeted,retweeted_author_id,retweeted_handle,retweeted_follower_count,mentioned_author_ids,mentioned_handles,hashtags,urls,media_keys,place_id
0,1131594960443199488,britney_201904,2022-02-28 09:34:44.627023-05:00,2022-02-28 09:34:44.627023-05:00,True,True,False,False,False,False,...,1130917791752757254,3042894016,Iesbwian,22760,None,None,None,None,None,None
1,1131594976750653440,britney_201904,2022-02-28 09:34:44.626921-05:00,2022-02-28 09:34:44.626921-05:00,True,True,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
2,1131589737955942405,britney_201904,2022-02-28 09:34:44.634058-05:00,2022-02-28 09:34:44.634058-05:00,True,True,False,False,False,False,...,None,None,None,None,None,None,None,None,None,None
3,1131594909469892610,britney_201904,2022-02-28 09:34:44.627125-05:00,2022-02-28 09:34:44.627125-05:00,True,True,False,False,False,False,...,1130917791752757254,3042894016,Iesbwian,22760,None,None,None,None,None,None
4,1131594812694511617,britney_201904,2022-02-28 09:34:44.627227-05:00,2022-02-28 09:34:44.627227-05:00,True,True,False,False,False,False,...,1130917791752757254,3042894016,Iesbwian,22760,None,None,None,None,None,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88037 entries, 0 to 88036
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   id                             88037 non-null  int64 
 1   event                          88037 non-null  object
 2   ts1                            88037 non-null  object
 3    ts2                           88037 non-null  object
 4   from_stream                    88037 non-null  bool  
 5   directly_from_stream           88037 non-null  bool  
 6   from_search                    88037 non-null  bool  
 7   directly_from_search           88037 non-null  bool  
 8   from_quote_search              88037 non-null  bool  
 9   directly_from_quote_search     88037 non-null  bool  
 10  from_convo_search              88037 non-null  bool  
 11  directly_from_convo_search     88037 non-null  bool  
 12  from_timeline_search           88037 non-null  bool  
 13  d

In [5]:
df.describe()

,id,author_id,conversation_id,author_follower_count,retweet_count,reply_count,like_count,quote_count
count,8.803700e+04,8.803700e+04,8.803700e+04,8.803700e+04,88037.000000,88037.000000,8.803700e+04,88037.000000
mean,1.128375e+18,2.840685e+17,1.128223e+18,5.871965e+04,613.836353,8.710633,1.643042e+02,9.802674
std,3.001479e+15,4.376297e+17,7.439132e+15,1.200196e+06,4047.938227,382.716992,8.811544e+03,470.460294
min,1.101535e+18,1.357300e+04,1.597973e+17,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000
25%,1.126285e+18,2.148085e+08,1.126257e+18,2.140000e+02,0.000000,0.000000,0.000000e+00,0.000000
50%,1.128714e+18,1.635683e+09,1.128709e+18,5.950000e+02,20.000000,0.000000,0.000000e+00,0.000000
75%,1.130296e+18,7.860916e+17,1.130281e+18,1.761000e+03,286.000000,0.000000,0.000000e+00,0.000000
max,1.134308e+18,1.134151e+18,1.134308e+18,1.143075e+08,415500.000000,81701.000000,1.368193e+06,76727.000000


In [6]:
df.isna().sum()

id                               0
event                            0
ts1                              0
 ts2                             0
from_stream                      0
directly_from_stream             0
from_search                      0
directly_from_search             0
from_quote_search                0
directly_from_quote_search       0
from_convo_search                0
directly_from_convo_search       0
from_timeline_search             0
directly_from_timeline_search    0
text                             0
lang                             0
author_id                        0
author_handle                    0
created_at                       0
conversation_id                  0
possibly_sensitive               0
reply_settings                   0
source                           0
author_follower_count            0
retweet_count                    0
reply_count                      0
like_count                       0
quote_count                      0
replied_to          

In [8]:
#How many tweets were posted containing the term on each day?
def tweets_per_day(term):
    query = f"""
    SELECT date(created_at) as tweet_date, COUNT(*) as tweet_count
    FROM tweets
    WHERE text LIKE '%{term}%'
    GROUP BY tweet_date
    ORDER BY tweet_date;
    """
    result = pd.read_sql_query(query, conn)
    return result

#How many unique users posted a tweet containing the term?
def unique_users(term):
    query = f"""
    SELECT COUNT(DISTINCT id) as unique_user_count
    FROM tweets
    WHERE text LIKE '%{term}%';
    """
    result = pd.read_sql_query(query, conn)
    return result

#How many likes did tweets containing the term get on average?
def average_likes(term):
    query = f"""
    SELECT AVG(like_count) as avg_likes
    FROM tweets
    WHERE text LIKE '%{term}%';
    """
    result = pd.read_sql_query(query, conn)
    return result

#Where (in terms of place IDs) did the tweets come from?
def tweet_locations(term):
    query = f"""
    SELECT place_id, COUNT(*) as tweet_count
    FROM tweets
    WHERE text LIKE '%{term}%'
    GROUP BY place_id
    ORDER BY tweet_count DESC;
    """
    result = pd.read_sql_query(query, conn)
    return result

#What times of day were the tweets posted at?
def tweets_time_of_day(term):
    query = f"""
    SELECT strftime('%H', created_at) as hour_of_day, COUNT(*) as tweet_count
    FROM tweets
    WHERE text LIKE '%{term}%'
    GROUP BY hour_of_day
    ORDER BY hour_of_day;
    """
    result = pd.read_sql_query(query, conn)
    return result

#Which user posted the most tweets containing the term?
def top_user(term):
    query = f"""
    SELECT id, COUNT(*) as tweet_count
    FROM tweets
    WHERE text LIKE '%{term}%'
    GROUP BY id
    ORDER BY tweet_count DESC
    LIMIT 1;
    """
    result = pd.read_sql_query(query, conn)
    return result


def main():
    # Ask the user for the term
    term = input("Enter the term you want to search for: ")

    print(f"\nSearching for tweets containing the term '{term}'...\n")
    
    print("Here are the results as per your search...")

    # Call each function and store the result
    tweets_day = tweets_per_day(term)
    unique_user_count = unique_users(term)
    avg_likes = average_likes(term)
    locations = tweet_locations(term)
    times_of_day = tweets_time_of_day(term)
    top_user_info = top_user(term)

    # Display results
    print("Number of tweets per day containing the term:\n", tweets_day)
    print("\nNumber of unique users posting tweets containing the term:\n", unique_user_count)
    print("\nAverage number of likes for tweets containing the term:\n", avg_likes)
    print("\nTop tweet locations (by place ID) for tweets containing the term:\n", locations)
    print("\nTweet counts per hour of the day for tweets containing the term:\n", times_of_day)
    print("\nUser who posted the most tweets containing the term:\n", top_user_info)


# Run the main function if the script is executed directly
if __name__ == "__main__":
    main()

Enter the term you want to search for: music

Searching for tweets containing the term 'music'...

Here are the results as per your search...
Number of tweets per day containing the term:
     tweet_date  tweet_count
0   2019-03-12            3
1   2019-04-06            1
2   2019-04-14            1
3   2019-04-16            1
4   2019-04-21            1
5   2019-04-24            1
6   2019-04-26            1
7   2019-04-27            3
8   2019-04-28           22
9   2019-04-29          118
10  2019-04-30          135
11  2019-05-01           71
12  2019-05-02           72
13  2019-05-03          103
14  2019-05-04           75
15  2019-05-05           65
16  2019-05-06           71
17  2019-05-07           64
18  2019-05-08           60
19  2019-05-09           70
20  2019-05-10          307
21  2019-05-11           78
22  2019-05-12           69
23  2019-05-13           50
24  2019-05-14           67
25  2019-05-15           99
26  2019-05-16           91
27  2019-05-17          162